In [99]:
#!/usr/bin/env python
# coding: utf-8

In [2]:
!pip install dask-expr
!pip install opencv-python
!pip install xarray
!pip install pydap
!pip install pyhdf
!pip install basemap
!pip install pygrib

### Paste into Terminal
sudo apt-get update
sudo apt-get install libgl1-mesa-glx

  Using cached dask_expr-1.0.10-py3-none-any.whl.metadata (2.4 kB)
  Using cached dask-2024.4.1-py3-none-any.whl.metadata (3.8 kB)
Using cached dask_expr-1.0.10-py3-none-any.whl (184 kB)
Using cached dask-2024.4.1-py3-none-any.whl (1.2 MB)
  Attempting uninstall: dask
    Found existing installation: dask 2024.3.0
    Uninstalling dask-2024.3.0:
      Successfully uninstalled dask-2024.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-ai 2.11.0 requires faiss-cpu, which is not installed.
distributed 2024.3.0 requires dask==2024.3.0, but you have dask 2024.4.1 which is incompatible.
  Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.2 MB)
  Using cached xarray-2024.3.0-py3-none-any.whl.metadata (1

In [10]:
popo = os.path.getsize('/home/sagemaker-user/inference/ifs')
lopo = os.path.getsize('/home/sagemaker-user/ifs-2')
print(popo,lopo)

1658880 1343488


In [4]:
import sklearn
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GroupKFold

import lightgbm as lgb

import pandas as pd

import warnings
warnings.resetwarnings()
warnings.filterwarnings("ignore", category=DeprecationWarning)#
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', category=sklearn.exceptions.ConvergenceWarning)
warnings.filterwarnings('ignore', category=sklearn.exceptions.FitFailedWarning)

from IPython.display import display
pd.set_option('display.max_columns', 100)

import cv2

import xarray as xr

import pydap

import h5py

from pyhdf.SD import SD, SDC

from mpl_toolkits.basemap import Basemap

import matplotlib as mpl

from joblib import Parallel, delayed

import random

import secrets

import datetime

import pygrib

import time

import zstandard as zstd

import tarfile

import io

import requests

import pickle

import subprocess

import json

import os

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.max_open_warning'] = 100

from collections import defaultdict

import numpy as np
pd.set_option('display.max_columns', 100)

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


In [102]:
zd = zstd.ZstdDecompressor()





SEED = datetime.datetime.now().microsecond


random.seed(datetime.datetime.now().microsecond); 
np.random.seed(datetime.datetime.now().microsecond)
run_label = secrets.token_hex(3)





dataset = 'tg'


ASSIM = True

In [103]:
labels = pd.read_csv('/home/sagemaker-user/data_{}/train_labels.csv'.format(dataset))
grid = pd.concat((
    pd.read_csv('/home/sagemaker-user/data_tg/grid_metadata.csv'),
    # pd.read_csv('data_pm/grid_metadata.csv')
)).drop_duplicates().reset_index(drop=True)

submission = pd.read_csv('/home/sagemaker-user/data_{}/submission_format.csv'.format(dataset))
files = pd.read_csv('/home/sagemaker-user/data_{}/{}_satellite_metadata{}.csv'.format(
                    dataset, *(('pm25', '') if dataset == 'pm'
                               else ('no2', '_0AF3h09'))))

labels['location'] = grid.set_index(
    'grid_id')['location'].reindex(labels.grid_id).values
labels['datetime'] = pd.to_datetime(labels.datetime)

submission['location'] = grid.set_index(
    'grid_id').location.reindex(submission.grid_id).values

# loc_dict = {'la': 'Los Angeles (SoCAB)', 'tpe': 'Taipei', 'dl': 'Delhi'}
loc_dict = {'la': 'Los Angeles (SoCAB)'}

In [104]:
coords = defaultdict(list)
for city, gcity in loc_dict.items():  # ['Delhi', 'Taipei', 'LA']:
    # gcity = [c for c in grid.location.unique() if 'Los A' in c][0] if city not in grid.location.unique() else city
    grid_points = grid[grid.location == gcity]
    for e in grid_points.itertuples():
        coords[city].append((e.grid_id, *np.array([(float(p.split(' ')[0]),
                                                    float(p.split(' ')[1]))
                                                   for p in e.wkt[10:].split(', ')[:4]]).mean(axis=0).round(4).tolist()))

In [105]:
# Clean Coords

def cleanDict(d):
    return {
        k: cleanDict(v) for k,
        v in d.items()} if isinstance(
        d,
        defaultdict) else d

assert sum([len(v) for c, v in coords.items()]) == grid.grid_id.nunique()
grid.grid_id.nunique()

coords = cleanDict(coords)

In [106]:
## Check/Validate 'ifs' files

# t = datetime.datetime(2016, 1, 1)
# # t = datetime.datetime(2020, 5, 26)
# while t < datetime.datetime.now() - datetime.timedelta(days = 1):
#     print(t)
#     for tag in ifs_tags:
#     # for tag in ifs_forecasts:# ifs_tags:#[:1]:
#         domain = 'ec.oper.an.sfc'
#         # domain = 'ec.oper.fc.sfc'
#         file =  '{}/{}/{}.{}.regn1280sc.{}.nc'.format(domain,
#                         datetime.datetime.strftime(t, '%Y%m'), 
#                         domain, tag, 
#                         datetime.datetime.strftime(t, '%Y%m%d') )
#         if file.split('/')[-1] not in existing_files:
#             sqs.send_message(
#                 QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-ifs',
#                 MessageBody=json.dumps({'file': file}
#             ) )
#         # else:
#         #     print(file, 'exists')
#     t += datetime.timedelta(days = 1)
#     # break;

In [107]:
fields = ['nitrogendioxide_tropospheric_column',
 'nitrogendioxide_tropospheric_column_precision',
 'air_mass_factor_troposphere',
 'air_mass_factor_total']

In [108]:
trop_scr_dir = '/home/sagemaker-user/tropomi_data/secure.txt'
with open(trop_scr_dir, 'r') as file:
    trop_secure = dict(e.split('=') for e in file.read().split('\n') if e)
download_dir = '/tmp/'

def loadFile(row):
    for i in range(1):
        try:
            values = {
                'email': trop_secure['username'],
                'passwd': trop_secure['password'],
                'action': 'login'}
            login_url = 'https://urs.earthdata.nasa.gov'
            ret = requests.post(login_url, data=values)
            if ret.status_code == 200:
                print("Login successful.")
            else:
                print("Bad Authentication")
                return None
        except Exception as e:
            print(e)
            time.sleep(i)
    zc = zstd.ZstdCompressor(level=15)
    os.makedirs(download_dir, exist_ok=True)
    filename, url, sz = [row[k]
                         for k in ['granule_id', 'us_url', 'granuleSize']]
    # print(filename, url, sz)
    file = '/tmp/' + filename
    try:
        print("Downloading", filename)
        if os.path.exists(file):
            print(f"{file} already exists. Skipping download.")
            pass
        with requests.get(url, cookies=ret.cookies,
                          allow_redirects=True, stream=True) as r:
            r.raise_for_status()
            with open(file, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    f.write(chunk)
        print(f"Downloaded and compressed {filename} to {file}")
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")
    return file

In [109]:
#  sklearn


#  warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning)#
# warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.filterwarnings('ignore', category=sklearn.exceptions.ConvergenceWarning)
# warnings.filterwarnings('ignore', category=sklearn.exceptions.FitFailedWarning)

In [110]:
def run(hdf, location, fine=True):
    zones = {}  # defaultdict(dict)
    sat_data = defaultdict(lambda: defaultdict(dict))
    hp = hdf['PRODUCT']
    lat = hp['latitude'][:][0]  # .values
    lon = hp['longitude'][:][0]  # .values
    for field in fields:
        v = hp[field][:][0]
        data = np.ma.masked_array(
            v, (v == v.max()) | (
                v == v.min())).clip(
            0, None)
        assert data.shape == lat.shape
        for grid_id, plon, plat in coords[location]:
            for r in ([0.07, 0.1, 0.14, 0.2, 0.3, 0.5, 1, 2]
                      if fine else [0.1, 0.25, 0.5, 1, 2, ]):
                if (grid_id, r) not in zones:
                    zones[(grid_id, r)] = (lat - plat) ** 2 + \
                        (lon - plon) ** 2 < r ** 2
                zone = zones[(grid_id, r)]
                ct = data[zone].count()
                m = data[zone].mean() if ct > (0 if 'fine' else 3) else np.nan
                s = data[zone].std() if ct >= 3 else np.nan
                sat_data[grid_id][field + '_mean{}'.format(r)] = m
                sat_data[grid_id][field + '_stdev{}'.format(r)] = s
                sat_data[grid_id][field + '_count{}'.format(r)] = ct
                if '2' in grid_id:  # .startswith('9'):
                    print(field, '_count{}'.format(r), ct, m, s)
    return sat_data

In [111]:
def runTropomi(row, fine=True):
    assert row['product'].startswith('tropomi')

    # Presumably, loadFile is a function you've defined elsewhere to load the
    # data file
    file = loadFile(row)
    hdf = h5py.File(file, 'r')

    # Assuming run is a function you've defined that processes the hdf file
    # based on the location and 'fine' parameter
    sat_data = run(hdf, row['location'], fine)
    output = row.copy()

    # Assuming cleanDict is a function you've defined to clean or process the
    # satellite data in some way
    output['d1'] = cleanDict(sat_data)

    # Compress the serialized output
    zc = zstd.ZstdCompressor(level=15)
    o = pickle.dumps(output)
    c = zc.compress(o)

    # Save the compressed data locally
    local_dir = "path/to/your/local/directory"  # Specify your directory here
    filename = 'aqi_tropomi{}-{}.zst'.format(
        '-fine' if fine else '', os.path.basename(file))
    local_path = os.path.join(local_dir, filename)
    with open(local_path, 'wb') as f:
        f.write(c)
    os.remove(file)
    return {
        'statusCode': 200,
        'len': len(o),
        'clen': len(c),
        'body': json.dumps('Success'),
        'localPath': local_path  # Optionally return the path where the file was saved
    }

In [112]:
# runTropomi(rows[-4], fine = True)





# rows[-1]


# %%time
# Parallel(os.cpu_count() #// 2
#             )(delayed(runTropomi)(row) for row in rows)








# d = pd.to_datetime(files[(files['product'] == 'misr')
#        & (files.location == 'la')].time_end)
# d = d.sort_values().reset_index(drop = True)
# (d.diff().dt.total_seconds() / (60*60*24)).iloc[:].plot()








# coords['LA']














# sqs.send_message(
#     QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-gfs25-extra',
#     MessageBody=json.dumps((2020, 1, 1, 6, 0))
# )


# t = datetime.datetime(2016, 10, 1)
# while t < datetime.datetime.now():
#     print(t)
#     for hr in range(0, 24, 6):
#         for fwd in (0, ):
#             sqs.send_message(
#                 QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-gfs25-extra',
#                 MessageBody=json.dumps((t.year, t.month, t.day, hr, fwd))
#             )
#     t += datetime.timedelta(days = 1)











# [e[0] for e in pickle.loads(zd.decompress(
#     s3.get_object(Bucket = 'projects-v',
#               Key = 'aqi/gfs-5/{}'.format('gfs.0p25.2016100218.f000.grib2'))['Body'].read()
# )) if 'LA' in e[1]]


# [e[0] for e in pickle.loads(
#     s3.get_object(Bucket = 'projects-v',
#               Key = 'aqi/gfs/{}'.format('gfs.0p25.2016100112.f000.grib2'))['Body'].read()
# ) if 'LA' in e[1]]


# [e for e in pickle.loads(
#     s3.get_object(Bucket = 'projects-v',
#               Key = 'aqi/gfs/{}'.format('gfs.0p25.2019100612.f000.grib2'))['Body'].read()
# ) if 'metre temp' in e[0]]











# grid.wkt


# grid.sort_values('location').wkt


# p = pygrib.open(# 'gfs.0p25.2020072500.f000.grib2')#'gfs.0p25.2020010100.f000.grib2')
#     # 'gfs.0p25.2017072500.f000.grib2'
#     'gfs.0p25.2017072500.f000.grib2'
# )

In [113]:
# cities = {
# 'Taipei': ( (121.5, 121.5), (25.0, 25) ),
# 'Delhi': ( (77.0, 77.25), (28.75, 28.5) ),
# 'LA': ((360-118.25, 360-117.75), (34.0, 34.0) )
# }

cities = {
    'LA': ((360 - 118.25, 360 - 117.75), (34.0, 34.0))
}

In [114]:
# for city in ['Delhi', 'Taipei', 'LA']:
for city in ['LA']:
    # Ensure the list exists for the city key
    coords.setdefault(city, [])

    gcity = [c for c in grid.location.unique() if 'Los A' in c][0] if city not in grid.location.unique() else city
    grid_points = grid[grid.location == gcity]
    for e in grid_points.itertuples():
        coords[city].append((e.grid_id, *np.array([(float(p.split(' ')[0]),
                                                    float(p.split(' ')[1]))
                                                   for p in e.wkt[10:].split(', ')[:4]]).mean(axis=0).tolist()))

### Load Saved Features

In [115]:
def loadFiles(file):
    zd = zstd.ZstdDecompressor()
    data = []
    with tarfile.TarFile(file, 'r') as tar:
        assim_files = tar.getnames()
        for file in assim_files:
            if len(file) > 0:
                f = tar.extractfile(file)
                data.append(pickle.loads(zd.decompress(f.read())))
    return data

In [116]:
# def listFiles(prefix, bucket = 'projects-v'): 
#     paginator = s3.get_paginator('list_objects_v2')
#     page_iterator = paginator.paginate(Bucket = bucket,
#                                        Prefix = prefix )
#     files = []
#     for page in page_iterator:
#         files.extend([e['Key'] for e in page['Contents']])
#     return files


# def listFiles(prefix):
#     return [os.path.join('cache/', prefix[4:], f) 
#             for f in os.listdir('cache/' + prefix[4:])]








# def loadData(file):
#     zd = zstd.ZstdDecompressor()
#     data = pickle.loads(zd.decompress(open(file, 'rb').read()))
#     return data

In [117]:
def flatten(l): return [e for s in l for e in s]

In [118]:
# loc_dict = {'la': 'Los Angeles (SoCAB)', 'tpe': 'Taipei', 'dl': 'Delhi'}
loc_dict = {'la': 'Los Angeles (SoCAB)'}

rloc_dict = {v: k for k, v in loc_dict.items()}

tz_dict = grid.groupby('location').tz.first().to_dict()

loc_tz_dict = {rloc_dict[k]: v for k, v in tz_dict.items()}
loc_tz_dict

{'la': 'Etc/GMT+8'}

In [119]:
# def downloadFile(file):
#     s3 = boto3.client('s3')
#     s3.download_file('projects-v', file, 'cache/' + file[4:])


# def downloadFiles(files):
#     for path in set([f.split('/')[1] for f in files]):
#         os.makedirs('cache/{}'.format(path), exist_ok = True)
                    
#     Parallel(os.cpu_count() * 3)(delayed(downloadFile)(file)
#                                      for file in files)

# downloadFiles(assim_files)

In [120]:
all_data_assim = loadFiles('/home/sagemaker-user/cache/assim.tar')

all_data_assim = flatten(all_data_assim)

len(all_data_assim)

132960

In [121]:
rt_dict = {}

def process(d, e):
    t = d['time_end']
    if t not in rt_dict:
        rt = (t + 30).astype('datetime64[m]')
        rt_dict[t] = rt
    rt = rt_dict[t]
    grid_data_assim[grid_id][rt].update(e)
    grid_data_assim[grid_id][rt].update({'location': d['location']})

In [122]:
grid_data_assim = defaultdict(lambda: defaultdict(dict))
for d in all_data_assim:
    for grid_id, e in d['d1'].items():
        process(d, e)

In [123]:
def processAssim(grid_id, v):
    df = pd.DataFrame.from_dict(v, orient= 'index')
    
    # tz
    assert df.location.nunique() == 1 
    tz = loc_tz_dict[df.location.unique()[0]]
    df.index = df.index.tz_localize('UTC').tz_convert(tz).floor('1d')
    df.index.name = 'datetime'
    df.drop(columns = 'location', inplace = True)
    df = df.groupby(df.index).mean()#.sort_index()#.head(10)    

    # fill
    t = pd.to_datetime(submission.datetime.min()).tz_convert(tz).floor('1d')
    tf = pd.to_datetime(submission.datetime.max()).tz_convert(tz).floor('1d')
    df_t = set(df.index)
    extra_times = []
    while t <= tf:
        if t not in df_t:
            extra_times.append(t)
        t += datetime.timedelta(days = 1)
    df_extra = pd.DataFrame(np.nan, index = pd.Series(extra_times), columns = df.columns )
    df = pd.concat((df, df_extra)).sort_index()
    
    
    # ewm     
    n = 1
    df_ewm1 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())
    df_ewm1.columns = [c + '_{}day'.format(n) for c in df.columns]

    n = 2
    df_ewm2 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())
    df_ewm2.columns = [c + '_{}day'.format(n) for c in df.columns]

    n = 5
    df_ewm5 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())
    df_ewm5.columns = [c + '_{}day'.format(n) for c in df.columns]
        
    # raw_df = df
    df = pd.concat((df_ewm1, df_ewm2, df_ewm5,
                   ), axis = 1)
    
    # compile
    df.index = df.index.tz_localize(None)
    df.insert(0, 'grid_id', grid_id)
    return df;

In [124]:
all_dfs = Parallel(os.cpu_count())(delayed(processAssim)(grid_id, v)
                                   for grid_id, v in grid_data_assim.items() )


/tmp/ipykernel_1134/3943265332.py:22: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
/tmp/ipykernel_1134/3943265332.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
/tmp/ipykernel_1134/3943265332.py:22: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
/tmp/ipykernel_1134/3943265332.py:22: FutureWarning: The behavio

In [125]:
assim = pd.concat(all_dfs)
assim.index.name = 'datetime'
assim = assim.reset_index().set_index(['datetime', 'grid_id'])

del all_data_assim, grid_data_assim, all_dfs

In [126]:
all_data_tropomi = loadFiles('/home/sagemaker-user/cache/tropomi-fine.tar')
len(all_data_tropomi)

1385

In [127]:
grid_data_tropomi = defaultdict(list)
for d in all_data_tropomi:
    for grid_id, e in d['d1'].items():
        # e = blend(e)#e.copy()
        e['datetime'] = d['time_end']
        e['location'] = d['location']
        grid_data_tropomi[grid_id].append(e)
    # break;

In [128]:
all_dfs = []
for grid_id, v in grid_data_tropomi.items():
    df = pd.DataFrame(v)

    # tz
    assert df.location.nunique() == 1 
    tz = loc_tz_dict[df.location.unique()[0]]
    df.datetime = pd.to_datetime(df.datetime).dt.tz_convert(tz).dt.floor('1d')
    df.drop(columns = 'location', inplace = True)

    # group
    for col in [c for c in df.columns if '_mean' in c]:
        ct = np.where(df[col].isnull(), 0, df[col.replace('_mean', '_count')] )
        df[col.replace('_mean', '_sum')] = df[col] * ct
        df[col.replace('_mean', '_count')] = ct
    df = df.groupby(df.datetime).sum()#.sort_index()#.head(10)
    

    for col in [c for c in df.columns if '_mean' in c]:
        df[col] = (df[col.replace('_mean', '_sum')] 
                                / df[col.replace('_mean', '_count')]
                    )
    # filter
    df = df[[c for c in df.columns if
             ('.' in  c) and 
             ( ( '_mean' in c ) or ('column_stdev' in c) )
            ]]
    
    # fill
    t = pd.to_datetime(submission.datetime.min()).tz_convert(tz).floor('1d')
    tf = pd.to_datetime(submission.datetime.max()).tz_convert(tz).floor('1d')
    df_t = set(df.index)
    extra_times = []
    while t <= tf:
        if t not in df_t:
            extra_times.append(t)
        t += datetime.timedelta(days = 1)
    df_extra = pd.DataFrame(np.nan, index = pd.Series(extra_times), columns = df.columns )
    df = pd.concat((df, df_extra)).sort_index()
        
    # clip
    sigma = 4
    high = df.ewm(span = 100).mean() + sigma * df.ewm(span = 100).std().fillna(100000)
    df = df.clip(0, None)
    df = np.minimum(df, high)
    
    # ewm     
    n = 1
    df_ewm1 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())
    df_ewm1.columns = [c + '_{}day'.format(n) for c in df.columns]

    n = 3
    df_ewm3 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())
    df_ewm3.columns = [c + '_{}day'.format(n) for c in df.columns]
    
    raw_df = df
    df = pd.concat((df_ewm1, df_ewm3, 
                   ), axis = 1)
    
    # compile
    df.index = df.index.tz_localize(None)
    df.insert(0, 'grid_id', grid_id)
    all_dfs.append(df)


tropomi = pd.concat(all_dfs)
tropomi.index.name = 'datetime'
tropomi = tropomi.reset_index().set_index(['datetime', 'grid_id'])


del all_data_tropomi, grid_data_tropomi, all_dfs

In [129]:
# %%time
# downloadFiles(ifs_files)





# def loadData(f):
#     s3 = boto3.client('s3')
#     zd = zstd.ZstdDecompressor()
#     data = pickle.loads(zd.decompress(s3.get_object(Bucket = 'projects-v', Key = f)['Body'].read()))    
#     return data








# ifs_files = sorted(listFiles('aqi/ifs/'));


# ifs_tags = [
#     '128_015_aluvp', '128_134_sp', '128_136_tcw', 
#     # '128_137_tcwv', 
# '128_164_tcc',	   
#  '128_165_10u',
# '128_166_10v',
#  '128_167_2t',
# '128_168_2d',	
# '128_206_tco3',  	
# '228_246_100u',	   
# '228_247_100v',  	   
# ]


# ifs_files = [f for f in ifs_files # if any(z in f for z in ifs_tags) 
#                     if '.oper.fc'  in f
#             ]


# len(ifs_files)#[::1000]








# def loadData(f):
#     s3 = boto3.client('s3')
#     data = s3.get_object(Bucket = 'projects-v', Key = f)['Body'].read() 
#     return data


# %%time
# all_data_ifs = Parallel(os.cpu_count() * 3)(delayed(loadData)(d) for d in 
#                         [f for f in ifs_files[::1] ])
# # data = flatten(all_data)





# %%time
all_data_ifs = loadFiles('/home/sagemaker-user/cache/ifs.tar')
len(all_data_ifs)




# with tarfile.open('cache/ifs.tar', "w") as tar:
#     for file, data in zip(ifs_files[:], all_data_ifs[:]):
#         t = tarfile.TarInfo(name = file)
#         t.size = len(data)
#         b = io.BytesIO(data)
#         b.seek(0)
#         tar.addfile(
#             t, b)
#         # print(b)

25484

In [130]:
grid_data_ifs = defaultdict(lambda: defaultdict(dict))
for d in all_data_ifs:
    for dt, v in d.items():
        for grid_id, e in v.items():
            # e['timezone'] = grid_tz_dict[grid_id]#d['location']
            grid_data_ifs[grid_id][dt[0]].update(e)

In [131]:
all_dfs = []
# def compileIfs(grid_id, v):
for grid_id, v in grid_data_ifs.items():
    df = pd.DataFrame(v).T
    df.index.name = 'datetime'
    # df.sort_index(inplace = True)
    df = df.reset_index()
    df.sort_values('datetime', inplace = True)
    
    # break;
    extra_rows = []
    t = df.datetime.min()
    timestamps = set(df.datetime)
    t_max = df.datetime.max()
    while t < t_max:#max(timestamps):#df.datetime.max():
        t_next = t + datetime.timedelta(seconds = 60 * 60 * 12) 
        if t_next not in timestamps:
            if grid_id == '1X116': print(t_next)
            extra = df[df.datetime == t].copy() #if not last else last.copy()
            extra.datetime = t_next
            extra_rows.append(extra)
            # last = extra
        # else:
            # last = None
        t = t_next;
    if len(extra_rows) > 1 and grid_id == '1X116': 
        print('{} extra rows'.format(len(extra_rows)))
    df = pd.concat((df, *extra_rows))
    
    df.set_index('datetime', inplace = True)    

    # df = addExtraRows(df.reset_index()).set_index('datetime')    
    df = df.ewm(span = 2 * 2).mean().astype(np.float32)
    df.columns = [c + '_{}day'.format(2) for c in df.columns]

        
    # compile
    df.index = df.index.tz_localize(None)
    df.drop(columns = [c for c in df.columns if 'vapour' in c 
                       or 'mean0.05' in c 
                       or 'mean0.2' in c
                       or 'mean1' in c
                       or 'roughness' in c
                       or 'albedo' in c
                      ], inplace = True)
    df.columns = ['ifs_'+ c for c in df.columns]
    df.insert(0, 'grid_id', grid_id)
    # return df;

    all_dfs.append(df)
    # break;
    # break;

# all_dfs = Parallel(#os.cpu_count() * 2
#                   1)(delayed(compileIfs)(grid_id, v) 
#                            for grid_id, v in grid_data_ifs.items())


ifs = pd.concat(all_dfs)
ifs.index.name = 'datetime'
ifs = ifs.reset_index().set_index(['datetime', 'grid_id'])

In [132]:
for i in [0, 10, -1]:
    df = all_dfs[i]
    df['grid_id'] = df['grid_id'].astype(str)
    grid['grid_id'] = grid['grid_id'].astype(str)  # Ensure grid_id is string
    try:
        location = grid.set_index('grid_id').location[df.grid_id.iloc[0]]
        print(location)
    except KeyError:
        print(f"Location for grid_id {df['grid_id'].iloc[0]} not found.")
        location = "Unknown Location"  # Default or placeholder location
    df_numeric = df.select_dtypes(include=[np.number])
    # plt.matshow(df_numeric.corr())
    # plt.colorbar()
    # plt.title(location)
    # plt.show()  # Ensure the plot is shown for each iteration
    # plt.matshow(df.corr()); plt.colorbar(); plt.title(str(location))

df.columns[1::6]

del all_data_ifs, grid_data_ifs, all_dfs

Los Angeles (SoCAB)
Los Angeles (SoCAB)
Los Angeles (SoCAB)


In [133]:
# gfs_files = listFiles('aqi/gfs-5/')


# len(gfs_files)


# with tarfile.open('cache/gfs-5.tar', "w") as tar:
#     tar.add('cache/gfs-5/', arcname=os.path.sep)


# %%time
# all_data_gfs = Parallel(os.cpu_count() * 3)(delayed(loadData)(d) for d in 
#                         [f for f in gfs_files if 'f000' in f])
# all_data_gfs = flatten(all_data_gfs)

In [134]:
# %%time
all_data_gfs = loadFiles('/home/sagemaker-user/cache/gfs-5.tar')
all_data_gfs = flatten(all_data_gfs)


len(all_data_gfs)

17640

In [135]:
points = defaultdict(lambda: defaultdict(list))
for e in all_data_gfs:# [e for e in h if e[1] == city]:
    t = datetime.datetime.strptime(e[0][-12:-2], '%Y%m%d%H')
    n = ':'.join(e[0].split(':')[1:2])
    city = e[1]
    
    arr = e[3] 
    arr2 = cv2.resize(arr, None, fx = 5, fy = 5,  )
    
    for grid_id, x, y in coords[city]:
        v = arr2[ int(round(( e[2][0][1] - y ) / 0.05 + 2)), 
            int(round( ( (x % 360) - e[2][1][0] )/0.05 + 2 ) ) ]
        points[grid_id][n].append((t, v, e[-1]))

In [136]:
def addExtraRows(df):
    extra_rows = []
    t = df.datetime.min()
    timestamps = set(df.datetime)
    t_max = df.datetime.max()
    last = pd.DataFrame()  # Initialize 'last' as an empty DataFrame here
    while t < t_max:  # Iterate until the max datetime
        t_next = t + datetime.timedelta(seconds=60 * 60 * 6)
        if t_next not in timestamps:
            # Check if 'last' is empty using '.empty'
            extra = df[df.datetime == t].copy() if not last.empty else last.copy()
            extra.datetime = t_next
            extra_rows.append(extra)
            last = extra
        else:
            # Reset 'last' to an empty DataFrame instead of None
            last = pd.DataFrame()
        t = t_next
    if len(extra_rows) > 0:
        print('{} extra rows'.format(len(extra_rows)))
    # Return the original DataFrame concatenated with any extra rows
    return pd.concat([df] + extra_rows, ignore_index=True)

In [137]:
all_dfs = []
for grid_point, grid_data in points.items():
    dfs = []
    for label, d in grid_data.items():
        if 'Volumetric soil moisture content' in label: continue;
        df = pd.DataFrame(d, 
                     columns = ['datetime', label + '_local', label + '_city'])
        df.sort_values('datetime', inplace = True)
        dfs.append(df.set_index('datetime'))
                
    df = pd.concat(dfs, axis = 1)
    df = addExtraRows(df.reset_index()).set_index('datetime')    
    df = df.ewm(span = 4 * 3).mean().astype(np.float32)
    df.columns = [c + '_{}day'.format(3) for c in df.columns]
    df['grid_id'] = grid_point
    all_dfs.append(df)
gfs = pd.concat(all_dfs)
gfs = gfs.reset_index().set_index(['datetime', 'grid_id'])
#
        # print(df.corr().iloc[0, 1], label)
        # break;

2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows
2962 extra rows


In [138]:
del all_data_gfs, points, all_dfs

In [139]:
labels['dayofweek'] = labels.datetime.dt.dayofweek.astype(np.int32)


labels['dayinyear'] = labels.datetime.dt.dayofyear.astype(np.int32)


labels.head()

,datetime,grid_id,value,location,dayofweek,dayinyear
0,2019-01-01 08:00:00+00:00,3A3IE,8.695000,Los Angeles (SoCAB),1,1
1,2019-01-01 08:00:00+00:00,3S31A,10.496667,Los Angeles (SoCAB),1,1
2,2019-01-01 08:00:00+00:00,7II4T,37.208333,Los Angeles (SoCAB),1,1
3,2019-01-01 08:00:00+00:00,8BOQH,9.791667,Los Angeles (SoCAB),1,1
4,2019-01-01 08:00:00+00:00,A2FBI,4.308333,Los Angeles (SoCAB),1,1


In [140]:
def addGFS(labels):
    ldo = pd.to_datetime(labels.datetime).dt.floor('6h').dt.tz_localize(None)
    df = pd.concat( ( labels, 
            * [2/3 * gfs.reindex([ldo
                                + datetime.timedelta(seconds = 60 * 60 * 12), 
                             labels.grid_id]).reset_index(drop = True) 
               + 1/3 * gfs.reindex([ldo
                                + datetime.timedelta(seconds = 60 * 60 * 18), 
                             labels.grid_id]).reset_index(drop = True) 
                  #for k, v in gfs_ewms.items() 
              ],                       
                      ), axis = 1)
    return df

# gfs.groupby(['datetime', 'grid_id']).nunique().mean(axis = 1).sort_values()[-50:]

def ifsoffset12(c): return c.replace('ifs', 'ifs12')
def ifsoffset0(c): return c.replace('ifs', 'ifs0')

In [141]:
def addIFS(labels):
    ldo = pd.to_datetime(labels.datetime).dt.floor('12h').dt.tz_localize(None)
    df = pd.concat( ( labels, 
            * [ifs.reindex([ldo
                                + datetime.timedelta(seconds = 60 * 60 * 12), 
                             labels.grid_id]).reset_index(drop = True).rename(
                                   columns = ifsoffset12)

                , 
               ifs.reindex([ldo
                                # + datetime.timedelta(seconds = 60 * 60 * 0)
                                    , 
                             labels.grid_id]).reset_index(drop = True).rename(
                   columns = ifsoffset0)
                  #for k, v in gfs_ewms.items() 
              ],                       
                      ), axis = 1)
    return df

In [142]:
def addSat(labels, sat):
    
    tza = labels.datetime.copy()
    for location in labels.location.unique():
        t = pd.to_datetime(labels.datetime).dt.tz_convert( tz_dict[location] )
        t = t.dt.floor('1d').dt.tz_localize(None)
        tza = np.where(labels.location == location, t, tza)
    tza = pd.to_datetime(pd.Series(tza, index = labels.index))

    df = pd.concat( ( labels, 
            sat.reindex([tza,
                             labels.grid_id]).reset_index(drop = True) 
                     
                      ), axis = 1)
    return df

In [143]:
all_data = addGFS(labels)
all_data = addIFS(all_data)
all_data = addSat(all_data, maiac if dataset == 'pm' else tropomi)
if ASSIM: all_data = addSat(all_data, assim)
# all_data = addSat(all_data, misr)


all_data.tail()


all_data.head()

,datetime,grid_id,value,location,dayofweek,dayinyear,Wind speed (gust)_local_3day,Wind speed (gust)_city_3day,Surface pressure_local_3day,Surface pressure_city_3day,2 metre temperature_local_3day,2 metre temperature_city_3day,2 metre specific humidity_local_3day,2 metre specific humidity_city_3day,2 metre relative humidity_local_3day,2 metre relative humidity_city_3day,Apparent temperature_local_3day,Apparent temperature_city_3day,10 metre U wind component_local_3day,10 metre U wind component_city_3day,10 metre V wind component_local_3day,10 metre V wind component_city_3day,Cloud water_local_3day,Cloud water_city_3day,Relative humidity_local_3day,Relative humidity_city_3day,Total ozone_local_3day,Total ozone_city_3day,ifs12_downward_uv_radiation_at_the_surface_mean0.1_2day,ifs12_downward_uv_radiation_at_the_surface_mean0.5_2day,ifs12_downward_uv_radiation_at_the_surface_mean2_2day,ifs12_downward_uv_radiation_at_the_surface_mean5_2day,ifs12_surface_pressure_mean0.1_2day,ifs12_surface_pressure_mean0.5_2day,ifs12_surface_pressure_mean2_2day,ifs12_surface_pressure_mean5_2day,ifs12_total_column_water_mean0.1_2day,ifs12_total_column_water_mean0.5_2day,ifs12_total_column_water_mean2_2day,ifs12_total_column_water_mean5_2day,ifs12_surface_sensible_heat_flux_mean0.1_2day,ifs12_surface_sensible_heat_flux_mean0.5_2day,ifs12_surface_sensible_heat_flux_mean2_2day,ifs12_surface_sensible_heat_flux_mean5_2day,ifs12_surface_latent_heat_flux_mean0.1_2day,ifs12_surface_latent_heat_flux_mean0.5_2day,ifs12_surface_latent_heat_flux_mean2_2day,ifs12_surface_latent_heat_flux_mean5_2day,ifs12_total_cloud_cover_mean0.1_2day,ifs12_total_cloud_cover_mean0.5_2day,...,so2_mean0.1_2day,so2_mean0.25_2day,so2_mean0.5_2day,so2_mean1_2day,so2_mean2_2day,co_mean0.1_2day,co_mean0.25_2day,co_mean0.5_2day,co_mean1_2day,co_mean2_2day,no2_mean0.1_2day,no2_mean0.25_2day,no2_mean0.5_2day,no2_mean1_2day,no2_mean2_2day,o3_mean0.1_2day,o3_mean0.25_2day,o3_mean0.5_2day,o3_mean1_2day,o3_mean2_2day,pm25_rh35_gcc_mean0.1_2day,pm25_rh35_gcc_mean0.25_2day,pm25_rh35_gcc_mean0.5_2day,pm25_rh35_gcc_mean1_2day,pm25_rh35_gcc_mean2_2day,so2_mean0.1_5day,so2_mean0.25_5day,so2_mean0.5_5day,so2_mean1_5day,so2_mean2_5day,co_mean0.1_5day,co_mean0.25_5day,co_mean0.5_5day,co_mean1_5day,co_mean2_5day,no2_mean0.1_5day,no2_mean0.25_5day,no2_mean0.5_5day,no2_mean1_5day,no2_mean2_5day,o3_mean0.1_5day,o3_mean0.25_5day,o3_mean0.5_5day,o3_mean1_5day,o3_mean2_5day,pm25_rh35_gcc_mean0.1_5day,pm25_rh35_gcc_mean0.25_5day,pm25_rh35_gcc_mean0.5_5day,pm25_rh35_gcc_mean1_5day,pm25_rh35_gcc_mean2_5day
0,2019-01-01 08:00:00+00:00,3A3IE,8.695000,Los Angeles (SoCAB),1,1,5.413679,4.403292,92828.007812,100089.90625,281.026093,286.235413,0.002560,0.002854,35.595852,30.025583,279.274139,286.091034,-2.047880,-1.76149,-1.687304,-0.21584,0.004267,0.005275,10.903458,12.244355,329.625122,332.235718,745171.8750,739461.6875,711426.5000,690866.3750,97174.710938,94290.960938,95142.187500,95286.648438,4.466324,3.908245,4.787787,6.222759,-1149612.625,-1676964.875,-1217090.750,-1097625.125,-1.568464e+06,-6.496657e+05,-2806355.0,-3111115.75,0.044663,0.039892,...,3.690123e-10,3.700938e-10,3.349736e-10,2.383904e-10,1.841198e-10,2.645851e-07,2.654458e-07,2.435152e-07,1.861290e-07,1.489865e-07,1.072726e-08,1.084041e-08,9.688059e-09,5.729664e-09,2.602033e-09,2.922568e-08,2.909989e-08,3.022860e-08,3.413607e-08,3.754369e-08,13.694448,13.684081,12.449080,9.186067,7.114766,4.287931e-10,4.265869e-10,3.683897e-10,2.396264e-10,1.726736e-10,3.293613e-07,3.280886e-07,2.926124e-07,2.129792e-07,1.612483e-07,1.407059e-08,1.402868e-08,1.211337e-08,6.980736e-09,3.080813e-09,2.672559e-08,2.674875e-08,2.853741e-08,3.329452e-08,3.720178e-08,21.200350,21.053583,18.760553,13.533575,9.749266
1,2019-01-01 08:00:00+00:00,3S31A,10.496667,Los Angeles (SoCAB),1,1,5.968803,4.403292,100819.125000,100089.90625,286.146179,286.235413,0.003239,0.002854,34.337368,30.025583,285.988281,286.091034,-2.845850,-1.76149,-0.686538,-0.21584,0.008124,0.005275,12.777002,12.2

In [144]:
x = all_data[[c for c in all_data.columns if c not in ['datetime', 'value']]].copy()
x.head(5)

,grid_id,location,dayofweek,dayinyear,Wind speed (gust)_local_3day,Wind speed (gust)_city_3day,Surface pressure_local_3day,Surface pressure_city_3day,2 metre temperature_local_3day,2 metre temperature_city_3day,2 metre specific humidity_local_3day,2 metre specific humidity_city_3day,2 metre relative humidity_local_3day,2 metre relative humidity_city_3day,Apparent temperature_local_3day,Apparent temperature_city_3day,10 metre U wind component_local_3day,10 metre U wind component_city_3day,10 metre V wind component_local_3day,10 metre V wind component_city_3day,Cloud water_local_3day,Cloud water_city_3day,Relative humidity_local_3day,Relative humidity_city_3day,Total ozone_local_3day,Total ozone_city_3day,ifs12_downward_uv_radiation_at_the_surface_mean0.1_2day,ifs12_downward_uv_radiation_at_the_surface_mean0.5_2day,ifs12_downward_uv_radiation_at_the_surface_mean2_2day,ifs12_downward_uv_radiation_at_the_surface_mean5_2day,ifs12_surface_pressure_mean0.1_2day,ifs12_surface_pressure_mean0.5_2day,ifs12_surface_pressure_mean2_2day,ifs12_surface_pressure_mean5_2day,ifs12_total_column_water_mean0.1_2day,ifs12_total_column_water_mean0.5_2day,ifs12_total_column_water_mean2_2day,ifs12_total_column_water_mean5_2day,ifs12_surface_sensible_heat_flux_mean0.1_2day,ifs12_surface_sensible_heat_flux_mean0.5_2day,ifs12_surface_sensible_heat_flux_mean2_2day,ifs12_surface_sensible_heat_flux_mean5_2day,ifs12_surface_latent_heat_flux_mean0.1_2day,ifs12_surface_latent_heat_flux_mean0.5_2day,ifs12_surface_latent_heat_flux_mean2_2day,ifs12_surface_latent_heat_flux_mean5_2day,ifs12_total_cloud_cover_mean0.1_2day,ifs12_total_cloud_cover_mean0.5_2day,ifs12_total_cloud_cover_mean2_2day,ifs12_total_cloud_cover_mean5_2day,...,so2_mean0.1_2day,so2_mean0.25_2day,so2_mean0.5_2day,so2_mean1_2day,so2_mean2_2day,co_mean0.1_2day,co_mean0.25_2day,co_mean0.5_2day,co_mean1_2day,co_mean2_2day,no2_mean0.1_2day,no2_mean0.25_2day,no2_mean0.5_2day,no2_mean1_2day,no2_mean2_2day,o3_mean0.1_2day,o3_mean0.25_2day,o3_mean0.5_2day,o3_mean1_2day,o3_mean2_2day,pm25_rh35_gcc_mean0.1_2day,pm25_rh35_gcc_mean0.25_2day,pm25_rh35_gcc_mean0.5_2day,pm25_rh35_gcc_mean1_2day,pm25_rh35_gcc_mean2_2day,so2_mean0.1_5day,so2_mean0.25_5day,so2_mean0.5_5day,so2_mean1_5day,so2_mean2_5day,co_mean0.1_5day,co_mean0.25_5day,co_mean0.5_5day,co_mean1_5day,co_mean2_5day,no2_mean0.1_5day,no2_mean0.25_5day,no2_mean0.5_5day,no2_mean1_5day,no2_mean2_5day,o3_mean0.1_5day,o3_mean0.25_5day,o3_mean0.5_5day,o3_mean1_5day,o3_mean2_5day,pm25_rh35_gcc_mean0.1_5day,pm25_rh35_gcc_mean0.25_5day,pm25_rh35_gcc_mean0.5_5day,pm25_rh35_gcc_mean1_5day,pm25_rh35_gcc_mean2_5day
0,3A3IE,Los Angeles (SoCAB),1,1,5.413679,4.403292,92828.007812,100089.90625,281.026093,286.235413,0.002560,0.002854,35.595852,30.025583,279.274139,286.091034,-2.047880,-1.76149,-1.687304,-0.21584,0.004267,0.005275,10.903458,12.244355,329.625122,332.235718,745171.8750,739461.6875,711426.5000,690866.3750,97174.710938,94290.960938,95142.187500,95286.648438,4.466324,3.908245,4.787787,6.222759,-1149612.625,-1676964.875,-1217090.750,-1097625.125,-1.568464e+06,-6.496657e+05,-2806355.0,-3111115.75,0.044663,0.039892,0.075970,0.208282,...,3.690123e-10,3.700938e-10,3.349736e-10,2.383904e-10,1.841198e-10,2.645851e-07,2.654458e-07,2.435152e-07,1.861290e-07,1.489865e-07,1.072726e-08,1.084041e-08,9.688059e-09,5.729664e-09,2.602033e-09,2.922568e-08,2.909989e-08,3.022860e-08,3.413607e-08,3.754369e-08,13.694448,13.684081,12.449080,9.186067,7.114766,4.287931e-10,4.265869e-10,3.683897e-10,2.396264e-10,1.726736e-10,3.293613e-07,3.280886e-07,2.926124e-07,2.129792e-07,1.612483e-07,1.407059e-08,1.402868e-08,1.211337e-08,6.980736e-09,3.080813e-09,2.672559e-08,2.674875e-08,2.853741e-08,3.329452e-08,3.720178e-08,21.200350,21.053583,18.760553,13.533575,9.749266
1,3S31A,Los Angeles (SoCAB),1,1,5.968803,4.403292,100819.125000,100089.90625,286.146179,286.235413,0.003239,0.002854,34.337368,30.025583,285.988281,286.091034,-2.845850,-1.76149,-0.686538,-0.21584,0.008124,0.005275,12.777002,12

In [154]:
# '7334C' - > '7F1D1'
# 'HANW9' - > 'WZNCR'
if '7F1D1' not in x['grid_id'].cat.categories:
    x['grid_id'] = x['grid_id'].cat.add_categories(['7F1D1'])

if 'WZNCR' not in x['grid_id'].cat.categories:
    x['grid_id'] = x['grid_id'].cat.add_categories(['WZNCR'])

if dataset == 'tg':
    np.random.seed(SEED)
    x.loc[(x.grid_id == '7334C') & (np.random.random(len(x)) < 0.1), 'grid_id'] = '7F1D1'
    x.loc[(x.grid_id == 'HANW9') & (np.random.random(len(x)) < 0.1), 'grid_id'] = 'WZNCR'


x.grid_id = x.grid_id.astype('category');
x.location = x.location.astype('category');

y = all_data.value.astype(np.float32)
d = all_data.datetime


with open('/home/sagemaker-user/cache/all_data_{}.pkl'.format(dataset), 'wb') as pckl_file:
    pickle.dump(all_data, pckl_file)

In [155]:
x.columns = [c.replace(':', '_') for c in x.columns]

# Verify no NaN values remain
#assert x.isnull().sum().sum() == 0
x.shape

(17473, 313)

In [156]:
# x.fillna(method = 'ffill')


#x.isnull().sum()


#x.isnull().sum().sort_values()

In [157]:
# Select only numerical columns
#numerical_cols = x.select_dtypes(include=['number']).columns
# Replace NaN with the mean in numerical columns only
#x[numerical_cols] = x[numerical_cols].fillna(x[numerical_cols].mean())

assert x.isnull().sum().sum() == 0

In [158]:
enet_params = {
    'alpha': [  1e-2, 3e-2, 1e-1, 0.3, 1, 3, ],
    'l1_ratio': [ 0.01, 0.03, 0.1, 0.2, 0.5, 0.8, 0.9, ],
}

In [159]:
class PurgedKFold():
    def __init__(self, n_splits=5, gap = 30):
        self.n_splits = n_splits
        self.gap = gap
        
    def get_n_splits(self, X, y = None, groups = None): return self.n_splits
    
    def split(self, X, y=None, groups=None):
        # Ensure 'groups' is a Series and convert to datetime if not already
        if not isinstance(groups, pd.Series):
            groups = pd.Series(groups)
        if not pd.api.types.is_datetime64_any_dtype(groups):
            groups = pd.to_datetime(groups)

        groups = groups.sort_values()
        X = X.reindex(groups.index)
        y = y.reindex(X.index)

        X, y, groups = sklearn.utils.indexable(X, y, groups)
        indices = np.arange(X.shape[0])

        n_splits = self.n_splits
        for i in range(n_splits):
            test_indices = indices[i * len(X) // n_splits: (i + 1) * len(X) // n_splits]
            
            # Extract test period
            test_period_min = groups.iloc[test_indices].min()
            test_period_max = groups.iloc[test_indices].max()

            # Define train indices with gap
            train_indices = indices[(groups < test_period_min - pd.Timedelta(days=self.gap)) | 
                                    (groups > test_period_max + pd.Timedelta(days=self.gap))]

            yield train_indices, test_indices

class RepeatedPurgedKFold():
    def __init__(self, n_splits = 5, n_repeats = 1, gap = None):
        self.n_splits = n_splits
        self.n_repeats = n_repeats
        self.gap = gap
        
    def get_n_splits(self, X, y = None, groups = None): 
        return self.n_splits * self.n_repeats + self.n_repeats * ( self.n_repeats - 1) // 2
    
    def split(self, X, y=None, groups=None):
        for i in range(self.n_repeats):
            for f in PurgedKFold(self.n_splits + i, gap = self.gap if self.gap else None).split(X, y, groups):
                yield f

In [160]:
def posRMSE(y, y_pred):
    return mean_squared_error(y, y_pred.clip(0, None) * 2/3 + y_pred * 1/3) ** 0.5
pRMSE = make_scorer(posRMSE, greater_is_better = False)
SCORING = pRMSE

In [161]:
COLUMN_WIPE = 0# len(x.columns) // 10

In [162]:
def runENet(drop_cols=[], verbose=1):
    all_y_enet = []
    all_y_pred_enet = []
    enet_clfs = []
    enet_scalers = []
    for location in x.location.unique():
        if verbose > 0:
            print(location)
        x_loc = x[x.location == location].drop(columns=drop_cols)
        y_loc = y.reindex(x_loc.index)
        d_loc = d.reindex(x_loc.index)
        folds = list(
            PurgedKFold(
                4 if dataset == 'pm' else 3).split(
                x_loc,
                y_loc,
                d_loc))
        folds += [(np.arange(0, len(x_loc)), [])] * 2
        for train_fold, test_fold in folds:
            y_preds = []
            for i in range(8):
                scaler = StandardScaler()
                clf = ElasticNet(
                    max_iter=50000,
                    tol=1e-3,
                    selection='random',
                    precompute=True,
                    random_state=datetime.datetime.now().microsecond)
                model = RandomizedSearchCV(clf, enet_params,
                                           scoring=SCORING,
                                           # 'neg_root_mean_squared_error',
                                           cv=RepeatedPurgedKFold(random.randrange(3, 6),
                                                                  n_repeats=random.randrange(
                                                                      2, 4),
                                                                  gap=random.randrange(60, 120)),
                                           n_iter=random.randrange(4, 7),
                                           random_state=datetime.datetime.now().microsecond)
                # subset = train_fold#[:s].tolist() + train_fold[s +
                # l:].tolist()
                l = random.randrange(0, len(train_fold) // 10)
                s = random.randrange(0, len(train_fold) - l)
                subset = train_fold[:s].tolist() + train_fold[s + l:].tolist()
                xt = x_loc.iloc[subset, 2:].copy()
                # for c in misr.columns:
                if dataset == 'pm':
                    sample_cols = [
                        c for c in xt.columns if any(
                            z in c for z in [
                                'precision',
                                'air_mass',
                                'stdev'] + [
                                'no2',
                                'so2',
                                'co',
                                'o3',
                                'pm25_rh35_gcc',
                                'ifs'])]
                    xt[random.sample(sample_cols, k=random.randrange(
                        len(sample_cols) * 1 // 3, len(sample_cols) * 2 // 3))] = 0
                if dataset == 'tg':
                    if random.random() < 0.5:
                        xt[[c for c in xt.columns if 'precision' in c]] = 0
                    # if random.random() < 0.5: xt[[c for c in xt.columns if 'mean1' in c or 'mean2' in c]] = 0
                # xt[misr.columns] = 0
                # fj_drop =
                # if random.random() < {'Delhi': 0.5, 'Taipei': 0.2}.get(location, 0):
                #     xt[[c for c in xt.columns if 'FineMode' in c or 'Injection' in c]] = 0
                # # if random.random() < {'Delhi': 0.2, 'Taipei': 0.1}.get(location, 0):
                # #     xt[[c for c in xt.columns if 'Optical' in c]] = 0
                # if random.random() < {'Delhi': 0.85, 'Taipei': 0.5}.get(location, 0.2):
                #     xt[[c for c in xt.columns if c.startswith('ifs')]] = 0
                # xt = x_loc.iloc[subset].copy()
                xt[random.choices(xt.columns[2:], k=int(
                    round(random.random() * COLUMN_WIPE)))] = 0
                if random.random() < 0.3:
                    xt['dayinyear'] = 0
                model.fit(
                    pd.DataFrame(
                        scaler.fit_transform(xt).astype(
                            np.float32),
                        xt.index,
                        xt.columns),
                    y_loc.iloc[subset],
                    groups=d_loc.iloc[subset])
                enet_clfs.append(model.best_estimator_)
                enet_scalers.append(scaler)
                if i == 0 and verbose > 0:
                    display(
                        pd.DataFrame(
                            model.cv_results_).sort_values('rank_test_score').drop(
                            columns='params'))
                if len(test_fold) > 0:
                    y_pred = pd.Series(model.predict(
                        pd.DataFrame(scaler.transform(x_loc.iloc[test_fold, 2:]).astype(np.float32),
                                     columns=x_loc.columns[2:])),  # .clip(0, None),
                        index=y_loc.iloc[test_fold].index)
                    y_preds.append(y_pred)
            if len(test_fold) > 0:
                y_pred = pd.concat(y_preds)
                y_pred = y_pred.clip(
                    0,
                    None).groupby(
                    y_pred.index).mean().clip(
                    0,
                    None)
                y_pred = (y_pred.groupby(y_pred.index).mean())  # *2/3
                # + y_pred.groupby(y_pred.index).median() * 1/3 )
                all_y_enet.append(y_loc.iloc[test_fold])
                all_y_pred_enet.append(y_pred)
                if verbose >= 1:
                    print(location, round(np.corrcoef(
                        y_pred, all_y_enet[-1])[0, 1], 4))
                if verbose >= 3:
                    all_y_enet[-1].reset_index(
                        drop=True).ewm(span=10).mean().plot()
                    plt.plot(y_pred.reset_index(drop=True))
                    plt.title(location)
                    plt.figure()
                    print((x_loc.iloc[subset, 2:].std().clip(1, 1) *
                           model.best_estimator_.coef_).sort_values())
    return all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers

In [163]:
all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers = runENet(verbose = 2)

Los Angeles (SoCAB)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,mean_test_score,std_test_score,rank_test_score
0,0.026967,0.002319,0.005921,0.000297,0.03,0.1,-13.206347,-10.438387,-12.392928,-13.649838,-11.919309,-9.816363,-13.305223,-12.432071,-17.599536,-10.326110,-9.014657,-13.869864,-12.330886,2.211464,1
2,0.028186,0.002302,0.006561,0.002669,0.01,0.1,-13.242156,-10.462364,-12.396563,-13.677541,-11.966232,-9.844755,-13.303409,-12.429545,-17.717157,-10.322463,-9.020458,-13.860058,-12.353558,2.231325,2
3,0.023066,0.005991,0.007155,0.002862,0.1,1,-13.132818,-10.786172,-13.393440,-13.371436,-11.751331,-9.885743,-14.541740,-13.487649,-12.520582,-10.279361,-9.761692,-15.412980,-12.360412,1.782225,3
1,0.019508,0.002561,0.005856,0.000148,0.2,3,-13.646703,-11.172047,-14.806535,-13.977876,-11.734162,-12.451897,-15.564945,-14.471654,-12.674583,-11.412148,-12.719842,-16.313853,-13.412187,1.588380,4
4,0.115386,0.024349,0.005739,0.003826,0.8,0.01,-15.877201,-24.195745,-13.155545,-15.129004,-24.364973,-9.911700,-14.640535,-12.279149,-22.894820,-11.365763,-9.197068,-16.333115,-15.778718,5.111962,5


Los Angeles (SoCAB) 0.7297


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,mean_test_score,std_test_score,rank_test_score
3,0.023574,0.000975,0.005870,0.000141,0.01,0.3,-11.936643,-11.808175,-14.076179,-12.407718,-10.806867,-9.873267,-15.432573,-12.334489,1.751061,1
2,0.029976,0.001541,0.005770,0.000136,0.01,0.1,-12.105749,-12.133740,-14.335490,-12.597767,-10.454230,-9.823215,-15.470966,-12.417308,1.843804,2
0,0.019675,0.002888,0.005996,0.000245,0.03,1,-12.327967,-11.417756,-14.059541,-12.754926,-11.369921,-10.504495,-15.431376,-12.552283,1.582537,3
1,0.130558,0.021297,0.004560,0.001089,0.9,0.01,-12.760099,-12.748056,-15.568634,-13.159027,-10.348283,-10.262058,-15.339121,-12.883611,1.951146,4
4,0.084383,0.014930,0.005781,0.000391,0.5,0.01,-13.727751,-12.734760,-15.912884,-13.987008,-10.273137,-10.128674,-15.299276,-13.151927,2.099811,5


Los Angeles (SoCAB) 0.7532


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,split11_test_score,mean_test_score,std_test_score,rank_test_score
4,0.038851,0.002077,0.005901,0.000256,0.8,0.03,-10.718849,-11.031767,-13.976767,-11.014452,-10.101296,-11.674500,-12.068467,-11.252166,-10.523952,-15.713971,-12.581864,-12.222986,-11.906753,1.527224,1
3,0.050869,0.005170,0.005846,0.000160,0.01,0.03,-10.975365,-10.769286,-13.989164,-11.417016,-10.008823,-11.753695,-12.216724,-11.530064,-11.179893,-15.224528,-12.617438,-12.317683,-11.999973,1.375583,2
2,0.022019,0.002006,0.005853,0.000181,0.1,1,-11.567001,-11.282190,-14.200776,-11.645234,-12.559392,-13.404575,-12.743569,-11.839645,-10.667462,-11.230596,-14.649744,-12.726991,-12.376431,1.181569,3
1,0.021625,0.001920,0.006148,0.001070,0.03,3,-11.926114,-11.938615,-14.631882,-12.164935,-13.088110,-14.089903,-13.215248,-12.366183,-10.945663,-11.857339,-15.325362,-13.177482,-12.893903,1.228602,4
0,0.020918,0.002709,0.005911,0.000218,0.9,1,-11.635697,-13.479916,-15.100815,-12.142216,-14.491558,-13.928085,-13.675353,-12.333540,-11.262761,-12.746856,-15.695604,-13.645627,-13.344836,1.309118,5


Los Angeles (SoCAB) 0.7527


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,mean_test_score,std_test_score,rank_test_score
3,0.053214,0.002297,0.006158,0.000185,0.9,0.03,-10.735449,-9.918701,-12.525054,-10.179056,-12.119157,-10.898745,-9.857062,-11.179561,-12.212202,-9.352404,-12.631734,-11.055375,1.114087,1
2,0.097573,0.003570,0.007058,0.002631,0.03,0.01,-11.740391,-9.932893,-12.689703,-10.101309,-12.375375,-11.328997,-9.906083,-11.072128,-12.386089,-9.398345,-12.533155,-11.224043,1.161361,2
0,0.038128,0.003373,0.006070,0.000109,0.9,0.1,-10.944166,-10.057457,-12.447962,-10.187962,-12.581966,-11.065490,-10.069867,-11.503831,-11.947136,-9.465533,-13.349070,-11.238222,1.183943,3
1,0.034407,0.001774,0.006045,0.000091,0.03,3,-12.199351,-11.330199,-14.044395,-11.115167,-13.820720,-12.392144,-11.190979,-13.676161,-12.719647,-10.510740,-14.551198,-12.504609,1.308265,4


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_l1_ratio,param_alpha,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,mean_test_score,std_test_score,rank_test_score
3,0.037002,0.002389,0.006349,0.000406,0.9,0.1,-10.470746,-11.449018,-10.855779,-12.032954,-10.809034,-9.842910,-12.490637,-9.888391,-12.831462,-11.185659,1.024508,1
0,0.106802,0.007219,0.006919,0.001906,0.03,0.01,-10.227232,-11.312087,-11.112385,-12.284174,-10.571796,-9.690306,-12.681188,-10.176833,-12.893714,-11.216635,1.101652,2
4,0.061957,0.001746,0.006421,0.000471,0.01,0.03,-10.362306,-11.414790,-11.102326,-12.088366,-10.701741,-9.836652,-12.749210,-10.172314,-12.691431,-11.235460,1.017776,3
5,0.032684,0.003152,0.006218,0.000170,0.01,1,-11.241577,-12.442193,-11.298455,-12.642646,-11.647386,-10.653244,-13.476714,-10.160541,-13.370315,-11.881452,1.101996,4
1,0.030749,0.002428,0.006093,0.000185,0.8,3,-12.075495,-14.234054,-12.707766,-14.766726,-12.438599,-11.739899,-15.224635,-11.749211,-15.470636,-13.378558,1.445880,5
2,0.031212,0.002882,0.006116,0.000142,0.9,3,-12.079839,-14.334567,-12.794480,-14.871002,-12.428140,-11.762199,-15.373491,-11.760903,-15.585699,-13.443369,1.495003,6


In [164]:
os.makedirs('clfs_{}'.format(dataset), exist_ok = True)


pickle.dump( (all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers), 
                     open('clfs_{}/enet_clfs_{}.pkl'.format(
                                dataset, run_label), 'wb'))

/tmp/ipykernel_1134/1637870185.py:4: ResourceWarning: unclosed file <_io.BufferedWriter name='clfs_tg/enet_clfs_c77c99.pkl'>
  pickle.dump( (all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers),


In [165]:
# for i in range(3):
#     pd.Series(np.mean([e.coef_ for e in 
#                    enet_clfs[len(enet_clfs) * i//3:len(enet_clfs) * (i + 1)//3]],
#         axis = 0), index = x.columns[2:]).sort_values().plot(kind = 'barh',
#                                                                 figsize = (10, x.shape[1]// 4),
#                                                                 title = list(x.location.unique())[i], )
#     plt.figure()

In [166]:
# print(round(np.corrcoef( pd.concat(all_y_pred_enet).clip(0, None),#.reindex(df.index), 
#                             pd.concat(all_y_enet)#.reindex(df.index)
#            )[0, 1], 3))

In [167]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef( pd.concat(all_y_pred_enet).clip(0, None).reindex(df.index), 
                pd.concat(all_y_enet).reindex(df.index))[0, 1]; cs.append(c)
    print(location, round(c, 3) )
print('\nBlend:', round(np.mean(cs), 3))

Los Angeles (SoCAB) 0.751

Blend: 0.751


In [168]:
lgb_params = {
    'n_estimators': np.arange(200, 400, 10) if dataset == 'pm' else np.arange(300, 600, 20),#[ 150, 200, 300, ],
    'learning_rate': np.arange(0.01, 0.04, 0.003) if dataset == 'pm' else np.arange(0.01, 0.061, 0.005),# [0.03, 0.05, 0.07, ],
    'num_leaves': np.arange(4, 30) if dataset == 'pm' else np.arange(10, 30),# [5, 7, 10, 15, 20,],
    'min_child_weight': np.arange(0.02, 0.1, 0.01),#[  0.1, 0.2, ],
    'min_child_samples': [ 140, 170, 200, 250, 300, 400, 500, 600, 700, 850, 1000, 1400, ]
                                    if dataset == 'pm' else
                         [ 30, 40, 50, 60, 80, 100, 120, 150, 170, 200, 300, 500, 700, ]  
    , 
    'reg_lambda': [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1,  ],
    'reg_alpha':  [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, ],
    'linear_tree': [True, ],
    'subsample': np.arange(0.4, 0.901, 0.05),#[0.3, 0.5,  0.8],
    'subsample_freq': [1],
    'colsample_bytree': np.arange(0.3, 0.71, 0.05) if dataset == 'pm' else np.arange(0.2, 0.81, 0.05), #[0.5, 0.8, ],#0.3, 0.5, 0.8],
    'colsample_bynode': np.arange(0.4, 1.01, 0.05)  if dataset == 'pm' else np.arange(0.2, 1.01, 0.05), #[0.5, 0.8, ],#0.3, 0.5, 0.8],
    'linear_lambda': [1e-3, 3e-3, 1e-2, 3e-2, 0.1,],
    # 'max_bins': np.arange(120, 400, 20),
    # 'min_data_in_bin': np.exp(np.arange(np.log(3), np.log(12), 0.1)).astype(int), #np.arange(2, 10),# [2, 3, 4, 5, 10],
                # [ 192, 
                 #255, 255, 384, 512],
    'min_data_per_group': [10, 20, 50, 100],
    'max_cat_threshold': [  8, 16, 32, ], 
    
    'cat_l2': [0.1, 1, 10],#     if dataset == 'pm' else [1e-3, 1e-2, 1e-1],
    'cat_smooth': [0.1, 1, 10],#      if dataset == 'pm' else [1e-3, 1e-2, 1e-1],
}


In [169]:
FAST = False
GAUSSIAN = 0.05 #if dataset == 'pm' else 0.1
COLUMN_WIPE = len(x.columns) // 5 #  if dataset == 'pm' else 10)

In [170]:
def runLGB(drop_cols=[], verbose=1, n_bags=8, ):
    all_y_lgb = []
    all_y_pred_lgb = []
    tidx = 0
    lgb_clfs = []
    lgb_scalers = []
    for location in x.location.unique():
        random.seed(datetime.datetime.now().microsecond)
        np.random.seed(datetime.datetime.now().microsecond)
        if FAST and location != 'Delhi':
            continue
        if verbose > 0:
            print(location)
        x_loc = x[x.location == location].drop(
            columns=drop_cols)  # .iloc[:, :4]
        y_loc = y.reindex(x_loc.index)
        d_loc = d.reindex(x_loc.index)
        folds = list(
            (PurgedKFold(4) if dataset == 'pm' else PurgedKFold(
                3, gap=20)) .split(
                x_loc, y_loc, d_loc))
        folds += [(np.arange(0, len(x_loc)), [])] * 3
        for train_fold, test_fold in folds:
            y_preds = []
            for i in range(n_bags):  # if location == 'Delhi' else 4):
                model = RandomizedSearchCV(lgb.LGBMRegressor(seed=datetime.datetime.now().microsecond,
                                                             # n_jobs = 2, #
                                                             # os.cpu_count()
                                                             ), lgb_params,
                                           cv=RepeatedPurgedKFold(random.randrange(3, 6),
                                                                  n_repeats=random.randrange(
                                                                      1, 3),
                                                                  gap=random.randrange(60, 120)),
                                           n_iter=random.randrange(3, 5),
                                           scoring=SCORING,
                                           n_jobs=-1,  # os.cpu_count(),
                                           random_state=datetime.datetime.now().microsecond)
                l = random.randrange(0, len(train_fold) // 10)
                s = random.randrange(0, len(train_fold) - l)
                subset = train_fold[:s].tolist() + train_fold[s + l:].tolist()
                xt = x_loc.iloc[subset].copy()
                xt.iloc[:, 2:] += (GAUSSIAN * random.random() *
                                   np.random.default_rng().standard_normal(size=xt.iloc[:, 2:].shape)
                                   * xt.iloc[:, 2:].std().values[None, :])
                for c in random.choices(xt.columns[2:], k=int(
                        round(random.random() * COLUMN_WIPE))):
                    xt[c] = 0
                if dataset == 'pm' and random.random() < 0.3:
                    xt['dayinyear'] = 0
                sample_cols = [
                    c for c in xt.columns if any(
                        z in c for z in [
                            'precision',
                            'air_mass',
                            'stdev'] + [
                            'no2',
                            'so2',
                            'co',
                            'o3',
                            'pm25_rh35_gcc',
                            'ifs'])]
                xt[random.sample(sample_cols, k=random.randrange(
                    len(sample_cols) * 1 // 3, len(sample_cols) * 2 // 3))] = 0
                # if random.random() < {'Delhi': 0.7, 'Taipei': 0.2}.get(location, 0):
                #     xt[[c for c in xt.columns if 'FineMode' in c or 'Injection' in c]] = 0
                # if random.random() < {'Delhi': 0.1, 'Taipei': 0.1}.get(location, 0):
                #     xt[[c for c in xt.columns if 'Optical' in c]] = 0
                # if random.random() < {'Delhi': 1.0, 'Taipei': 0.2}.get(location, 0.2):
                #     xt[[c for c in xt.columns if c.startswith('ifs')]] = 0
                # if random.random() < {'Delhi': 0.9,}.get(location, 0.12):
                #     xt[[c for c in xt.columns if any(z + '_' in c for z in
                #          ['no2', 'so2', 'co', 'o3', 'pm25_rh35_gcc',  ] ) ]] = 0
                scaler = StandardScaler()
                xt.iloc[:, 2:] = scaler.fit_transform(
                    xt.iloc[:, 2:]).astype(np.float32)
                # for i in xt, y_loc.iloc[subset], d_loc.iloc[subset]:
                #     if i
                y_loc_subset = y_loc.iloc[subset].astype(np.int64)
                d_loc_subset = d_loc.iloc[subset].astype(np.int64) # //1e9
                print(f'xt = {xt}')
                print(f'y_loc.iloc[subset] = {y_loc_subset}')
                print(f'd_loc.iloc[subset] = {d_loc_subset}')
                model.fit(xt, y_loc_subset, groups=d_loc_subset,
                         )
                lgb_clfs.append(model.best_estimator_)
                lgb_scalers.append(scaler)
                if len(test_fold) > 0:
                    xtst = x_loc.iloc[test_fold].copy()
                    xtst.iloc[:, 2:] = scaler.transform(
                        xtst.iloc[:, 2:]).astype(np.float32)
                    y_pred = pd.Series(
                        model.predict(xtst),
                        index=y_loc.iloc[test_fold].index)
                    y_preds.append(y_pred)
                df = pd.DataFrame(model.cv_results_).sort_values(
                    'rank_test_score').drop(columns='params')
                if i == 0 and verbose > 1:  # df.mean_test_score.min() < -0:
                    display(df)
                    print()
            if len(test_fold) > 0:
                y_pred = pd.concat(y_preds)
                y_pred = (y_pred.groupby(y_pred.index).mean())  # * 2/3
                # + y_pred.groupby(y_pred.index).median() * 1/3 )
                all_y_lgb.append(y_loc.iloc[test_fold])
                all_y_pred_lgb.append(y_pred)
                if verbose >= 3:
                    all_y_lgb[-1].reset_index(
                        drop=True).ewm(span=10).mean().plot()
                    plt.plot(y_pred.reset_index(drop=True), linewidth=0.8)
                    try:
                        plt.plot(
                            all_y_pred_enet[tidx].clip(
                                0, None).reset_index(
                                drop=True))
                    except BaseException:
                        pass
                    plt.title(location)
                    plt.figure()
                tidx += 1
                if verbose > 0:
                    print(location, round(np.corrcoef(
                        y_pred, all_y_lgb[-1])[0, 1], 4))
    return all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers

In [171]:
all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers = runLGB(verbose = 2, )

Los Angeles (SoCAB)
xt =       grid_id             location  dayofweek  dayinyear  \
6332    3A3IE  Los Angeles (SoCAB)  -1.014567   0.514593   
6333    3S31A  Los Angeles (SoCAB)  -0.985511   0.505688   
6334    7II4T  Los Angeles (SoCAB)  -0.989280   0.537462   
6335    8BOQH  Los Angeles (SoCAB)  -0.994714   0.514277   
6336    A2FBI  Los Angeles (SoCAB)  -1.026122   0.498132   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.034460   1.071960   
17469   X5DKW  Los Angeles (SoCAB)   1.006428   1.055612   
17470   Z0VWC  Los Angeles (SoCAB)   1.025005   1.104559   
17471   ZP1FZ  Los Angeles (SoCAB)   1.002153   1.065309   
17472   ZZ8JF  Los Angeles (SoCAB)   1.003234   1.080178   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
6332                       0.051916                     0.056277   
6333                       0.026088                     0.034691   
6334                       0.058331               

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow 

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
1,6.890861,0.414336,0.233992,0.017331,1,0.8,0.1,0.01,22,480,100,0.08,50,16,True,0.03,0.03,0.7,0.4,0.1,1,-11.239849,-8.565344,-10.363054,-10.056082,1.113229,1
3,1.077809,0.311777,0.108738,0.003224,1,0.5,0.0001,0.1,16,540,10,0.04,500,32,True,0.01,0.05,0.5,0.7,10,10,-11.363771,-9.170572,-11.339981,-10.624775,1.028323,2
0,4.219241,0.369012,0.239594,0.019229,1,0.9,0.1,0.00001,10,500,50,0.04,40,8,True,0.03,0.045,0.6,1.0,0.1,10,-12.075554,-9.486905,-10.430038,-10.664166,1.069700,3
2,2.503578,0.800709,0.198759,0.028894,1,0.85,0.1,0.00001,16,520,20,0.08,500,16,True,0.03,0.06,0.6,0.75,10,0.1,-11.428188,-9.019246,-11.676778,-10.708071,1.198484,4



xt =       grid_id             location  dayofweek  dayinyear  \
6332    3A3IE  Los Angeles (SoCAB)  -1.021679   0.552523   
6333    3S31A  Los Angeles (SoCAB)  -1.064523   0.536710   
6334    7II4T  Los Angeles (SoCAB)  -0.955429   0.505623   
6335    8BOQH  Los Angeles (SoCAB)  -1.032432   0.532994   
6336    A2FBI  Los Angeles (SoCAB)  -0.961785   0.453686   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.002130   1.099279   
17469   X5DKW  Los Angeles (SoCAB)   0.967505   1.025573   
17470   Z0VWC  Los Angeles (SoCAB)   1.001215   1.071794   
17471   ZP1FZ  Los Angeles (SoCAB)   0.968252   1.063162   
17472   ZZ8JF  Los Angeles (SoCAB)   0.970244   1.073496   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
6332                       0.094431                     0.039153   
6333                       0.056569                     0.072353   
6334                       0.074975                     0.030482   
6

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,split10_test_score,mean_test_score,std_test_score,rank_test_score
2,6.475724,0.646086,0.120654,0.018021,1,0.7,0,0.001,25,320,100,0.06,80,16,True,0.1,0.045,0.65,0.65,0.1,10,-11.170922,-7.005288,-8.142892,-7.426509,-12.588072,-11.571449,-7.663702,-6.178452,-7.926790,-7.197930,-16.006136,-9.352558,2.911632,1
0,8.461029,0.570434,0.135795,0.022678,1,0.6,0.001,0.00001,26,340,20,0.06,60,16,True,0.003,0.025,0.8,0.85,10,1,-14.416040,-6.791719,-8.252987,-6.788823,-12.215618,-13.819062,-7.642347,-5.790627,-7.866321,-6.701756,-13.229749,-9.410459,3.132946,2
3,4.672887,0.784252,0.153415,0.042632,1,0.55,0.001,0.01,13,520,100,0.02,170,16,True,0.01,0.04,0.65,1.0,0.1,1,-11.805368,-6.947956,-9.117531,-7.278406,-16.077673,-12.641748,-7.649075,-6.723155,-7.559220,-7.197783,-13.760384,-9.705300,3.132478,3
1,2.557211,0.283835,0.112405,0.012658,1,0.4,0.0001,0.00001,24,340,20,0.08,50,32,True,0.01,0.015,0.2,0.3,0.1,0.1,-13.746123,-9.017344,-8.635739,-9.304297,-17.134572,-17.142331,-9.624381,-7.957251,-8.736274,-9.485420,-19.517076,-11.845528,4.028806,4


...


xt =       grid_id             location  dayofweek  dayinyear  \
0       3A3IE  Los Angeles (SoCAB)  -0.993628  -1.796133   
1       3S31A  Los Angeles (SoCAB)  -0.964203  -1.806174   
2       7II4T  Los Angeles (SoCAB)  -1.005219  -1.817647   
3       8BOQH  Los Angeles (SoCAB)  -0.993513  -1.827519   
4       A2FBI  Los Angeles (SoCAB)  -0.988805  -1.811901   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.021039   1.304361   
17469   X5DKW  Los Angeles (SoCAB)   1.019315   1.289469   
17470   Z0VWC  Los Angeles (SoCAB)   1.029279   1.287385   
17471   ZP1FZ  Los Angeles (SoCAB)   1.029721   1.299842   
17472   ZZ8JF  Los Angeles (SoCAB)   1.020509   1.295509   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
0                          3.434366                     3.408127   
1                          4.266706                     3.437569   
2                          1.095166                     3.455260   
3

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,4.653761,0.584716,0.158227,0.038195,1,0.4,0.00001,0,17,320,20,0.03,60,32,True,0.001,0.045,0.75,0.4,0.1,10,-11.805066,-6.589734,-8.871956,-7.366915,-8.724792,-8.671693,1.783140,1
0,3.066365,0.184489,0.224352,0.019223,1,0.75,0.1,0.0001,10,460,20,0.06,500,16,True,0.003,0.01,0.3,0.85,10,10,-8.719194,-7.053078,-10.337427,-7.730686,-9.861584,-8.740394,1.238917,2
2,1.829130,0.437644,0.109676,0.016553,1,0.65,0.1,0.001,13,300,100,0.04,500,16,True,0.001,0.01,0.3,0.4,10,1,-9.789285,-7.966658,-11.312136,-8.838596,-11.348409,-9.851017,1.338407,3



xt =       grid_id             location  dayofweek  dayinyear  \
0       3A3IE  Los Angeles (SoCAB)  -0.999143  -1.656283   
1       3S31A  Los Angeles (SoCAB)  -0.952810  -1.694471   
2       7II4T  Los Angeles (SoCAB)  -0.970625  -1.656196   
3       8BOQH  Los Angeles (SoCAB)  -0.970083  -1.674457   
4       A2FBI  Los Angeles (SoCAB)  -0.986748  -1.687008   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   0.979303   1.379663   
17469   X5DKW  Los Angeles (SoCAB)   1.030922   1.422569   
17470   Z0VWC  Los Angeles (SoCAB)   0.990865   1.346336   
17471   ZP1FZ  Los Angeles (SoCAB)   1.036895   1.435351   
17472   ZZ8JF  Los Angeles (SoCAB)   1.010771   1.412812   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
0                          3.562289                     3.591297   
1                          4.399307                     3.558658   
2                          1.097368                     3.562667   
3

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample_freq,param_subsample,param_reg_lambda,param_reg_alpha,param_num_leaves,param_n_estimators,param_min_data_per_group,param_min_child_weight,param_min_child_samples,param_max_cat_threshold,param_linear_tree,param_linear_lambda,param_learning_rate,param_colsample_bytree,param_colsample_bynode,param_cat_smooth,param_cat_l2,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,8.691764,0.584195,0.210887,0.028234,1,0.6,0.001,0.001,13,440,50,0.09,40,32,True,0.003,0.01,0.75,0.7,1,0.1,-7.341347,-6.223598,-8.941974,-7.385667,-8.801626,-7.738842,1.015454,1
2,6.042452,0.837978,0.270306,0.058330,1,0.8,0.001,0.00001,22,380,10,0.09,300,16,True,0.1,0.055,0.55,0.8,10,0.1,-9.414204,-6.425394,-8.678122,-7.317312,-7.983025,-7.963611,1.039128,2
0,11.711507,1.095230,0.472910,0.045423,1,0.65,0,0.0001,18,560,20,0.08,170,8,True,0.1,0.05,0.8,0.7,10,10,-8.524512,-6.748477,-8.739427,-7.357676,-8.542585,-7.982535,0.786591,3
3,3.624895,0.504478,0.195018,0.010401,1,0.6,0.00001,0.01,26,440,20,0.05,60,8,True,0.1,0.055,0.2,0.8,10,0.1,-11.394401,-6.789405,-9.665547,-8.109352,-9.522054,-9.096152,1.554504,4



xt =       grid_id             location  dayofweek  dayinyear  \
0       3A3IE  Los Angeles (SoCAB)  -1.041041  -1.691418   
1       3S31A  Los Angeles (SoCAB)  -1.015517  -1.628923   
2       7II4T  Los Angeles (SoCAB)  -0.950231  -1.684354   
3       8BOQH  Los Angeles (SoCAB)  -0.926052  -1.666106   
4       A2FBI  Los Angeles (SoCAB)  -1.004675  -1.646592   
...       ...                  ...        ...        ...   
17468   WT52R  Los Angeles (SoCAB)   1.026435   1.232671   
17469   X5DKW  Los Angeles (SoCAB)   0.984650   1.293489   
17470   Z0VWC  Los Angeles (SoCAB)   1.010019   1.229181   
17471   ZP1FZ  Los Angeles (SoCAB)   1.007604   1.270972   
17472   ZZ8JF  Los Angeles (SoCAB)   1.023227   1.348553   

       Wind speed (gust)_local_3day  Wind speed (gust)_city_3day  \
0                          3.494969                     3.536592   
1                          4.302358                     3.482778   
2                          1.206946                     3.475956   
3

In [183]:
with open(f'clfs_{dataset}/lgb_clfs_{run_label}.pkl', 'wb') as file:
    pickle.dump((all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers), file)

In [184]:
# # print(round(np.corrcoef( pd.concat(all_y_pred_enet).clip(0, None), pd.concat(all_y_lgb))[0, 1], 3)  )
# print(round(np.corrcoef( pd.concat(all_y_pred_lgb).clip(0, None), pd.concat(all_y_lgb))[0, 1], 3) )
# print(round(np.corrcoef( pd.concat(all_y_pred_lgb).clip(0, None)
#                   +  0.1  * pd.concat(all_y_pred_enet).clip(0, None)
#                   , pd.concat(all_y_lgb))[0, 1], 4) )
#  # without weather ~0.79 lgb

In [185]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef( pd.concat(all_y_pred_lgb).clip(0, None).reindex(df.index) 
                    + 0.1 * pd.concat(all_y_pred_enet).clip(0, None).reindex(df.index) 
                , pd.concat(all_y_lgb).reindex(df.index))[0, 1]; cs.append(c)
    print(location, 
          round(c, 3) )
print('\nBlend: ', round(np.mean(cs), 3))

len(lgb_clfs)

Los Angeles (SoCAB) 0.537

Blend:  0.537


48

In [177]:
submission = pd.read_csv('/home/sagemaker-user/data_{}/submission_format.csv'.format(dataset))


submission['dayinyear'] = pd.to_datetime(submission.datetime).dt.dayofyear
submission['dayofweek'] = pd.to_datetime(submission.datetime).dt.dayofweek


submission['location'] = grid.set_index('grid_id').location.reindex(submission.grid_id).values
submission['location'] = submission.location.astype('category')
submission['grid_id'] = submission.grid_id.astype(x.grid_id.dtype)#'category')


submission = addGFS(submission)
submission = addIFS(submission)
submission = addSat(submission, maiac if dataset == 'pm' else tropomi)
if ASSIM: submission = addSat(submission, assim)


with open(f'/home/sagemaker-user/cache/submission_{dataset}.pkl', 'wb') as file:
    pickle.dump(submission, file)

xs = submission[x.columns]

submission.location.value_counts()

clf_path = 'clfs_{}'.format(dataset)
all_clfs = os.listdir(clf_path)

In [178]:
# enet_tuples = [pickle.load(open( os.path.join(clf_path, f), 'rb'))
#                    for f in all_clfs if 'enet_clfs' in f] if dataset == 'pm' else []
# lgb_tuples = [pickle.load(open( os.path.join(clf_path, f), 'rb'))
#                    for f in all_clfs if 'lgb_clfs' in f]

In [179]:
enet_tuples = [(all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers)]
lgb_tuples = [(all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers)]


len(lgb_tuples)


len(enet_tuples)

1

### Labels

In [180]:
y_true = pd.concat(flatten([e[0] for e in lgb_tuples]))
y_true = y_true.groupby(y_true.index).mean()


y_pred_lgb = pd.concat(flatten([e[1] for e in lgb_tuples]))
y_pred_lgb =  y_pred_lgb.groupby(y_pred_lgb.index).mean() #* 2/3
                  # + y_pred_lgb.groupby(y_pred_lgb.index).median() * 1/3)


if dataset == 'pm':
    y_pred_enet = pd.concat(flatten([e[1] for e in enet_tuples]))
    y_pred_enet = ( y_pred_enet.groupby(y_pred_enet.index).mean() )
                      # + y_pred_enet.groupby(y_pred_enet.index).mean() * 1/3)


cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef( y_pred_lgb.clip(0, None).reindex(df.index) 
                    + 1/5 * ( y_pred_enet.clip(0, None).reindex(df.index) if dataset == 'pm' else 0), 
                y_true.reindex(df.index))[0, 1]; cs.append(c)
    print(location, 
          round(c, 3) )
print('\nBlend: ', round(np.mean(cs), 3))

Los Angeles (SoCAB) 0.511

Blend:  0.511


In [181]:
# [c.n_features_in_ for c in enet_scalers]


lgb_clfs, lgb_scalers, enet_clfs, enet_scalers = [], [], [], []
for i in range(3):
    lgb_clfs.extend(flatten([e[2][i * len(e[2]) //3 : (i + 1) * len(e[2]) // 3]
                                    for e in lgb_tuples]))
    lgb_scalers.extend(flatten([e[3][i * len(e[3]) //3 : (i + 1) * len(e[3]) // 3]
                                    for e in lgb_tuples]))
    enet_clfs.extend(flatten([e[2][i * len(e[2]) //3 : (i + 1) * len(e[2]) // 3]
                                    for e in enet_tuples]))
    enet_scalers.extend(flatten([e[3][i * len(e[3]) //3 : (i + 1) * len(e[3]) // 3]
                                    for e in enet_tuples]))

In [212]:
### Missing Values
non_zero_nulls = x_loc.isnull().sum()
for column, null_count in non_zero_nulls.items():
    if null_count != 0:
        print(f"{column}: {null_count} null values")

Wind speed (gust)_local_3day: 7183 null values
Wind speed (gust)_city_3day: 7183 null values
Surface pressure_local_3day: 7183 null values
Surface pressure_city_3day: 7183 null values
2 metre temperature_local_3day: 7183 null values
2 metre temperature_city_3day: 7183 null values
2 metre specific humidity_local_3day: 7183 null values
2 metre specific humidity_city_3day: 7183 null values
2 metre relative humidity_local_3day: 7183 null values
2 metre relative humidity_city_3day: 7183 null values
Apparent temperature_local_3day: 7183 null values
Apparent temperature_city_3day: 7183 null values
10 metre U wind component_local_3day: 7183 null values
10 metre U wind component_city_3day: 7183 null values
10 metre V wind component_local_3day: 7183 null values
10 metre V wind component_city_3day: 7183 null values
Cloud water_local_3day: 7183 null values
Cloud water_city_3day: 7183 null values
Relative humidity_local_3day: 7183 null values
Relative humidity_city_3day: 7183 null values
Total ozon

In [211]:
len(lgb_clfs)
assert x_loc.isnull().sum().sum() == 0

lgb_ys = []
for clf_idx, clf in enumerate(lgb_clfs):
    x_loc = xs[xs.location == x.location.unique()[0]].copy()  # Always use the first (and only) location
    x_loc.iloc[:, 2:] = lgb_scalers[clf_idx].transform(x_loc.iloc[:, 2:])
    lgb_ys.append(pd.Series(clf.predict(x_loc), index=x_loc.index))
    lgb_y_combined = pd.concat(lgb_ys, axis=0)
enet_ys = []
for clf_idx, clf in enumerate(enet_clfs):
    x_loc = xs[xs.location == x.location.unique()[0]].copy().iloc[:,2:]
    enet_ys.append(pd.Series(clf.predict(
        pd.DataFrame(enet_scalers[clf_idx].transform(x_loc), 
                     columns = x_loc.columns)), index = x_loc.index))
enet_ys = pd.concat(enet_ys)
lgb_y = lgb_y_combined.groupby(lgb_y_combined.index).mean().clip(0, None)  # .sort_values()
enet_y = enet_ys.groupby(enet_ys.index).mean().clip(0, None)  # .sort_values()

# plt.scatter(enet_y, lgb_y, s= 0.1)
print(round(np.corrcoef(enet_y, lgb_y)[0,1], 4) )#, s= 0.1)

f = 8 if dataset == 'pm' else 100000  # lgb_y
ys = (lgb_y * (f - 1) / f + enet_y * 1 / f)

ys.name = 'value'


out = pd.concat((submission[['datetime', 'grid_id']], ys.reindex(submission.index)), axis = 1)
out

dayofweek                                  0
dayinyear                                  0
Wind speed (gust)_local_3day            7183
Wind speed (gust)_city_3day             7183
Surface pressure_local_3day             7183
Surface pressure_city_3day              7183
2 metre temperature_local_3day          7183
2 metre temperature_city_3day           7183
2 metre specific humidity_local_3day    7183
2 metre specific humidity_city_3day     7183
dtype: int64


ValueError: Input X contains NaN.
ElasticNet does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# out[pd.to_datetime(out.datetime) < datetime.datetime(2018, 1, 1)]


# if dataset == 'pm' and ASSIM:
#     pivot = '2018-01-10'
#     b1 = pd.read_csv('submissions_pm/blend1.csv')
#     mix = pd.concat((
#         b1[b1.datetime < pivot],
#         out[out.datetime >= pivot] ) )
#     assert out.shape == mix.shape
#     out = mix

In [ ]:
# for g in out.grid_id.unique():
#     out[out.grid_id == g].set_index(pd.to_datetime(out[out.grid_id == g].datetime)).value.plot(
#         marker = '.', linewidth = 0.1,)# markersize = 3)
# plt.xlim(pd.to_datetime(submission.datetime).min(), 
#          pd.to_datetime(submission.datetime).min() + datetime.timedelta(days = 400));


# for g in out.grid_id.unique():
#     out[out.grid_id == g].set_index(pd.to_datetime(out[out.grid_id == g].datetime)).value.plot(
#         marker = '.', linewidth = 0.1,)# markersize = 3)
# plt.xlim(datetime.datetime(2020, 10, 15), datetime.datetime(2021, 9, 1));

In [ ]:
# if dataset == 'pm':
#     sp = pd.read_csv('submissions_pm/lgb_baseline.csv')
#     sp2 = pd.read_csv('submissions_pm/lgb_2.csv')
#     g1 = pd.read_csv('submissions_pm/first_gfs.csv')
#     m1 = pd.read_csv('submissions_pm/maiac.csv')
#     m2 = pd.read_csv('submissions_pm/maiac2.csv')
#     m3 = pd.read_csv('submissions_pm/maiac3.csv')
#     b1 = pd.read_csv('submissions_pm/blend1.csv')
#     b1f = pd.read_csv('submissions_pm/blend1f.csv')
# else:
#     sp = pd.read_csv('../submissions_tg/third_gfs.csv') 
#     s1 = pd.read_csv('../submissions_tg/sat1.csv') 
#     s2 = pd.read_csv('../submissions_tg/sat2.csv') 
#     s4 = pd.read_csv('../submissions_tg/sat4.csv') 
#     a1 = pd.read_csv('../submissions_tg/assim1.csv') 
#     n = pd.read_csv('../submissions_tg/new.csv') 
#     st1 = pd.read_csv('../submissions_tg/stack1.csv') 
#     n2 = pd.read_csv('submissions_tg/new.csv')


# if dataset == 'pm':
#     # plt.scatter(sp.value, out.value, s= 0.1);
#     # plt.scatter(sp2.value, out.value, s= 0.1);
#     # plt.scatter(g1.value, out.value, s= 0.1);
#     plt.scatter(m1.value, out.value, s= 0.1);
#     plt.scatter(m2.value, out.value, s= 0.1);
#     plt.scatter(m3.value, out.value, s= 0.1);
#     plt.scatter(b1.value, out.value, s= 0.1);
#     plt.scatter(b1f.value, out.value, s= 0.1);
# else:
#     plt.scatter(sp.value, out.value, s= 0.1);
#     plt.scatter(s1.value, out.value, s= 0.1);
#     plt.scatter(s2.value, out.value, s= 0.1);
#     plt.scatter(s4.value, out.value, s= 0.1);
#     plt.scatter(a1.value, out.value, s= 0.1);
#     plt.scatter(n.value, out.value, s= 0.1);
#     plt.scatter(n2.value, out.value, s= 0.1)
#     plt.scatter(st1.value, out.value, s= 0.1)

In [ ]:
# if dataset == 'pm': 
#     print(np.corrcoef((g1.value, m1.value, m2.value, m3.value, b1.value, b1f.value, out.value)).round(4))
# else:
#     print(np.corrcoef((sp.value, s1.value, s2.value, s4.value, a1.value, n.value, n2.value, st1.value, out.value)).round(4))


In [ ]:
out[out.datetime < '2021-04-01']

In [ ]:
os.makedirs('submissions_{}'.format(dataset), exist_ok = True)
out.to_csv('submissions_{}/new.csv'.format(dataset), index = False)